## Annotation ingestion notebook

Given a pair of strings representing a well formatted list of base companies and a list
of annotated peers respectively, this notebook processes them into an easy machine-understandable dictionary that can be used for evaluation.

In [1]:
from collections import defaultdict
from elasticsearch import Elasticsearch, NotFoundError
from elasticsearch_dsl import Search
import numpy as np
import ssl
from elasticsearch.connection import create_ssl_context
from elasticsearch_dsl import Q
from graphene import ObjectType, String, Int, ID, Float, List, Schema
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

HOST = "host.docker.internal"
PORT = "9200"
SSL = False
INDEX = "company"

if SSL:
    ssl_context = create_ssl_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
else:
    ssl_context = None

client = Elasticsearch(
    hosts=[{'host': HOST, 'port': PORT}],
    indices=[INDEX],
    scheme="https" if SSL else "http",
    ssl_context=ssl_context)

In [2]:
region_to_currency = {
    "europe": "EUR",
    "asia pacific": "MYR",
}

In [3]:
import requests

headers = {
  "X-API-Key": "ONAI_API_KEY",
  "X-Stack-Host": "Development",
  "X-Request-Id": "-1",
  "X-Stack-User": "Anonymous User"
}

get_conversion_rates_query = '''
query{
  currencyConversionRates(keys:
    [
      {sourceCurrency: "MYR",
        targetCurrency:"EUR",
        spotRate: "2019-11-30"
      },
      {sourceCurrency: "USD",
        targetCurrency:"EUR",
        spotRate: "2019-11-30"
      },
    ]
  )
  {
    sourceCurrency{
      code
    }
    rate
  }
}
'''

res = requests.post('https://data-services.onai.cloud/api/', 
                            json={'query': get_conversion_rates_query}, 
                            headers=headers).json()['data']['currencyConversionRates']
conversion_rates_to_eur = {
    el["sourceCurrency"]["code"]: el["rate"]
    for el in res
}

In [4]:
base_companies_str = '''
Noble Foods Limited 	Noble Foods Limited supplies eggs to retailers in the United Kingdom. It offers shell eggs; egg products, such as scrambled, boiled, and liquid eggs; desserts, including chocolate puds and patisserie; poultry products, including hens, chickens, and broiler breeders; and diets for pullets, laying hens, broiler chickens, and turkeys, as well as pig feeds. The company was founded in 1920 and is based in Witney, United Kingdom.	UK	Europe	GBP 344mn	402	Consumer Staples
Voicevale Ltd	Voicevale Ltd distributes food products in Europe and China. The company distributes nuts, dried fruits, seeds/superfood, coffee, and organic products. It also sources, stores, and supplies vine fruits and tropical fruits, as well as fruit and nut pastes, and freeze dried products. The company was founded in 1981 and is based in London, United Kingdom with additional offices in Paris, France; Hamburg, Germany; Beijing, China; Istanbul, Turkey; and La Paz, Bolivia.	UK	Europe	GBP 264mn	309	Consumer Staples
"R. Twining and Company Limited"	R. Twining and Company Limited produces and sells packaged teas. The company offers tea in various categories, such as black, flavoured black, fruit and herbal, green, Darjeeling, Assam, white, decaffeinated, organic, and breakfast teas, as well as chai, redbush/rooibos, and fairtrade teas and infusions. It also offers gifts that include hampers and gift sets, compartment boxes, filled jars, British themed gifts, union jack gift boxes, discovery collection, tea caddies, signature blends, and gift boxes and bags. In addition, the company offers tea ware product that include teapots, mugs, teacups and saucers, serve ware products, serving accessories, storage, and tea glasses; and confectionery products, such as biscuits and cakes, marmalade and preserves, chocolate, hot chocolate and malt drinks, coffee, sugar sticks, and syrups. It serves customers worldwide. The company offers its products through wholesale stockiests, as well as online. R. Twining and Company Limited was founded in 1706 and is based in London, United Kingdom. R. Twining and Company Limited operates as a subsidiary of Twining Crosfield and Company Ltd.	UK	Europe	GBP 234mn	274	Consumer Staples
Typhoo Tea Limited	Typhoo Tea Limited manufactures traditional and decaffeinated tea and also offers green and white varieties and a fruit selection. The company was founded in 1903 and is based in Wirral, United Kingdom. Typhoo Tea Limited operates as a subsidiary of Apeejay Tea Ltd.	UK	Europe	GBP 70mn	82	Consumer Staples
Thurn Produkte GmbH	Thurn Produkte GmbH manufactures liquid detergents and cleaning products. The company is based in Neunkirchen-Seelscheid, Germany.	Germany	Europe	EUR 189mn	189	Consumer Staples
Henkel Central Eastern Europe GmbH	Henkel Central Eastern Europe GmbH, manufactures and sells laundry and home care products. The company offers liquid detergents, fabric softeners, all-purpose detergents, hand and dishwashing detergents, and polish detergents. It sells its products in Austria and internationally. The company was founded in 1927 and is based in Vienna, Austria. Henkel Central Eastern Europe GmbH operates as a subsidiary of Henkel AG & Co. KGaA.	Austria	Europe	EUR 57mn	57	Consumer Staples
Ringana GmbH	Ringana GmbH produces natural cosmetics and freshness cosmetics. The company offers face care, special care, personal hygiene, sun care, washing hygiene, and decoration types of cosmetics. It also provides RINGANA WELLNESS PACKS that comprises phytoactive fruits, vegetables, and herbs; and WELLNESSCAPS, including beauty and hair, pump, immu, fem, cerebro, mascu, move, meta, hydro, and sun types. Ringana GmbH was founded in 1993 and is based in Hartberg, Austria.	Austria	Europe	EUR 62mn	62	Consumer Staples
ROMA Friseurbedarf Robert Maurer GmbH	ROMA Friseurbedarf Robert Maurer GmbH manufactures and supplies hair care and styling products. The company was founded in 1949 and is based in Vösendorf, Austria.	Austria	Europe	EUR 20mn	20	Consumer Staples
Sanmex International Limited	Sanmex International Limited manufacturers and markets household and personal care aerosol products. It offers air care, personal care, cleaning, furniture care, laundry care products, household, and car care products, including body sprays, anti perspirant, air fresheners, odor neutralizers, furniture polishes, and insecticides and general household products. The company sells its products through retailers, wholesalers, chemists, and market stalls in the United Kingdom and internationally. Sanmex International Limited was founded in 1932 and is based in Glasgow, United Kingdom. As of November 12, 2015, Sanmex International Limited operates as a subsidiary of Barony Universal Products Plc.	UK	Europe	EUR 10mn	10	Consumer Staples
PM-International AG	PM-International AG manufactures and distributes health, beauty, and wellness products. It offers nutritional supplements, cosmetics, and skin care products; and products for hobby and professional athletes. It distributes its products online; and through distributors or its direct sales centers. PM-International AG was formerly known as PM-International GmbH. The company was founded in 1993 and is based in Speyer, Germany with locations in Europe and internationally.	Germany	Europe	EUR 160mn	160	Consumer Staples
Feintechnik GmbH Eisfeld	Feintechnik GmbH Eisfeld manufactures and supplies shaving systems, disposable razors, and razor blades for man and woman. It also manufactures rezors under private labels. The company serves customers in Germany and internationally. Feintechnik GmbH Eisfeld was founded in 1920 and is based in Eisfeld, Germany. As of January 20, 2014, Feintechnik GmbH Eisfeld operates as a subsidiary of HF Global, Inc.	Germany	Europe	EUR 78mn	78	Consumer Staples
Vendor International B.V.	Vendor International B.V. develops, produces, and supplies toilet hygiene products. The company offers hand drying solutions that include IQ hand towel dispensers, folded hand towel dispensers, and autocut paper dispensers; hand cleaning solutions, such as soap/foam soap dispensers for everyday use or industrial soap dispensers for heavy-duty situations; horizontal and vertical toilet roll dispensers; air fresheners; ladies’ hygiene solutions comprising sanitary disposal bins, sanitary bag dispensers, and mini wall bins; paper products, including scrim-reinforced paper rolls, cassettes, and toilet papers; and other toilet and washroom solutions, which include waste bins, toilet seat cleaners, toilet brushes, sanitizers, floor/wall stands, urinal screens, and center-feed dispensers. It also provides towel cassettes, toilet paper, soap and foam soap, industrial hand cleansers, air freshener refills, folded hand towels, wiping paper, instant disinfectants, and other refills. It offers its products through a network of distributors in the Netherlands and internationally. The company was founded in 1961 and is based in Tilburg, the Netherlands with sales offices in Belgium, Germany, France, and Switzerland. Vendor International B.V. is a former subsidiary of Rentokil Initial (1896) Limited.	Netherlands	Europe	EUR 18mn	18	Consumer Staples
ApolloCorp Inc.	ApolloCorp Inc. manufactures and distributes private label and control label personal care products. It provides liquid and bar soaps, sanitizers, body washes, hair care products, skin care products, baby products, specialty bath products, promotion products, nutraceuticals, and gift baskets and seasonal items. The company serves customers in the United States, Mexico, Central and South America, the Middle East, Europe, and Asia. ApolloCorp Inc. is based in Toronto, Canada. 	US	United States and Canada	USD 110mn	99	Consumer Staples
The Honest Company, Inc.	The Honest Company, Inc. designs and manufactures bundles, baby, bath and body, cleaning, health and wellness, collective, and gift products. It offers diapers and wipes bundle, essentials bundle, health and wellness bundle, diapers, wipes, healing balm, diaper rash cream, organic baby powder, soothing bottom wash, organic breathe easy rub, training pants, organic shave oil, discovery set, shampoo and body wash, conditioner, conditioning detangler, sunscreen, sunscreen spray, sunscreen stick, bug spray, laundry detergent, laundry packs, dryer cloth, multi-surface cleaner, bathroom cleaner, glass and window cleaner, floor cleaner, stain remover, organic breathe easy rub, prenatal multi-vitamin, baby and toddler multi powder, kid's multi-vitamin, DHA/omega-3, kid's immunity boost, immunity defense, convertible crib with toddler rail, dresser with changer, lunch box, kid's sunglasses, diaper caddy, wood toy, bear hat, organic cotton blanket, gift cards, diaper cakes, surfboards sun and fun gift set, housewarming gift set, baby arrival gift set, and bath time gift set. The company sells its products through a network of retailers; and online. The company was incorporated in 2011 and is based in Santa Monica, California	US	United States and Canada	USD 109mn	98	Consumer Staples
Royal Paper Converting, Inc."	Royal Paper Converting, Inc. manufactures and distributes consumer, commercial, and industrial paper products. Its products include bathroom, facial, and pocket tissues; hard wound and multi-fold kitchen towels; and luncheon and dinner table and dispenser napkins. The company also provides paper and moist wipes for janitorial and foodservice segments. It sells its products through retailers, such as grocery, convenience store, and club accounts; secondary wholesale trade; and dollar and discount stores in the United States and Mexico. The company was founded in 1992 and is based in Phoenix, Arizona"	US	United States and Canada	USD 109mn	99	Consumer Staples
Freudenberg Household Products, Inc.	Freudenberg Household Products, Inc. engages in the manufacture and distribution of household cleaning products. The company offers sponge, flat, spring, and bathroom mops; angle, straight, and corn brooms; bathroom and basic scourers; household and disposable gloves; and dust pans, cloths, buckets, systems, and refills, as well as electrical cleaning products, such as rechargeable brooms and dust mops. It serves customers through retailers in Canada and internationally. The company is based in Laval, Canada. Freudenberg Household Products, Inc. operates as a subsidiary of Freudenberg & Co.	US	United States and Canada	USD 105mn	95	Consumer Staples
Jacobs Industries, Incorporated"	Jacobs Industries, Incorporated, through its subsidiaries, offer health and personal care products, including soap and detergents, specialty cleaners, toothpaste, and other similar products. The company was founded in 1977 and is based in Hopkins, Minnesota."	US	United States and Canada	USD 103mn	92	Consumer Staples
"Albon Engineering & Manufacturing Plc"	Albon Engineering & Manufacturing Plc engages in the design, machining, and assembly of engine components for automotive and diesel manufacturers worldwide. Its products include engine bearing caps for passenger car producers; connecting rods for motorcycles and automotive vehicles, as well as for marine and off highway applications; engine cylinder blocks for agricultural and marine applications; cylinder heads for automotive and heavy industrial applications; and flywheel assemblies for passenger cars and heavy trucks. The company specializes in the machining and assembly of engine components for automotive and diesel manufacturers in Europe, the United States, and internationally. The company was founded in 1970 and is based in Rochford, United Kingdom	UK	Europe	EUR 45mn	45	Auto Components
Benteler SGL Composite Technology GmbH	Benteler SGL Composite Technology GmbH designs and manufactures fiber composite components. It offers car parts, including engine covers, tail exclusion parts, and front spoilers. Benteler-SGL-Automotive-Composite GmbH & Co. KG was formerly known as Fischer Composite Technology GmbH and changed its name in February, 2009 following its acquisition by Benteler-SGL-Automotive-Composite GmbH & Co. KG.The company is based in Ried im Innkreis, Austria. Benteler SGL Composite Technology GmbH operates as a subsidiary of Benteler-SGL-Automotive-Composite GmbH & Co. KG.	Austria	Europe	EUR 54mn	54	Auto Components
Burg Design GmbH	Burg Design GmbH designs and manufactures automobile equipment and accessories. The company produces both interior and exterior components including cockpits and door parts, side mouldings, and mirror linings using different foil technologies and special design variants essentially for vehicle sales to OEM but also to a number of other target groups in the after-market sector such as general representatives and importers. The company was founded in 1977 and is based in Steyr, Austria. As of August 10, 2016, Burg Design GmbH operates as a subsidiary of LEONHARD KURZ Stiftung & Co. KG. 	Austria	Europe	EUR 21mn	21	Auto Components
Continental Automotive Austria Gmbh	Continental Automotive Austria Gmbh provides products and services for the automotive industry such as brake systems, sensorics, passive safety, and systems and components for hybrid and electric drives. The company was formerly known as Siemens VDO. The company was incorporated in 2000 and is based in Vienna, Austria. Continental Automotive Austria Gmbh operates as a subsidiary of Continental AG.	Austria	Europe	EUR 32mn	32	Auto Components
Linamar Antriebstechnik GmbH	Linamar Antriebstechnik GmbH manufactures precision engine, transmission, and driveline components and assemblies. It manufactures hydro-formed camshafts for engines. It was formerly known as Salzgitter Antriebstechnik GmbH & Co. KG and changed its name to Linamar Antriebstechnik GmbH in June 2003. The company is based in Crimmitschau, Germany. As of June 01, 2003, Linamar Antriebstechnik GmbH operates as a subsidiary of Linamar Corporation.	Germany	Europe	EUR 95mn	95	Auto Components
BBS GmbH	BBS GmbH develops and manufactures light-alloy wheels for original equipment manufacturer, motorsport, and branded aftermarket applications worldwide. It offers forged line, performance line, and design line range of products. It offers its products through dealerships. BBS GmbH was formerly known as BBS Kraftfahrzeugtechnik AG, OEM & Aftermarket Business and changed its name to BBS GmbH in March, 2012. The company was founded in 2012 and is based in Schiltach, Germany with additional locations in Herbolzheim, Germany; and Braselton, Georgia. As of July 2, 2015, BBS GmbH operates as a subsidiary of NICE Holdings Co., Ltd.	Germany	Europe	EUR 91mn	91	Auto Components
PRETTL Kabelkonfektion GmbH	PRETTL Kabelkonfektion GmbH develops and produces cable sets and wire harnesses for the automotive industry. It offers coils and modules, seats, seat adoptions, seat belt tightening systems, and seat modules, as well as wiring harnesses for ABS and emission control systems, plastic parts for navigation equipment and radios, and brush card systems for electric motors. The company was founded in 1996 and is based in Pfullingen, Germany. PRETTL Kabelkonfektion GmbH operates as a subsidiary of Prettl Group.	Germany	Europe	EUR 88mn	88	Auto Components
Greiner Perfoam GmbH	Greiner Perfoam GmbH develops and produces functional trim parts for vehicle interiors and trunks, as well as customized acoustic components for passenger and engine compartments for premium automobile manufacturers. Its interior products include convertible top compartments, loading space flooring, loading space covers, parcel shelves, and permanent and self-supporting parcel shelves. The company’s acoustics products comprise high expansion foam absorbers, vibration absorbers, and acoustic encapsulations. It also offers special products, such as pedestrian protection products and roller blind encapsulations. Greiner Perfoam GmbH was founded in 2000 and is based in Enns, Austria. The company has manufacturing sites in Linz, Austria; Tábor, Czech Republic; Wangen, Germany; Neukirchen/Pleisse, Germany; and China. Greiner Perfoam GmbH operates as a subsidiary of Greiner Holding AG. 	Austria	Europe	EUR 77mn	77	Auto Components
Otto Zimmermann Maschinen- und Apparatebau GmbH	Otto Zimmermann Maschinen- und Apparatebau GmbH develops, manufactures, and sells vehicle components for car manufacturers and the spare parts market worldwide. Its products include standard brake discs and brake drums, sport brake discs, Formula Z brake discs, brake discs for light commercial vehicles, brake kits, coated brake discs, coated brake discs for brake drums, brake pads, and mounted brake shoe kits. Otto Zimmermann Maschinen- und Apparatebau GmbH is based in Sinsheim, Germany.	Germany	Europe	EUR 77mn	77	Auto Components
FINOBA AUTOMOTIVE GmbH	FINOBA AUTOMOTIVE GmbH manufactures and delivers power-train, structural, and chassis cast-aluminum components for the automotive industry. The company offers cast aluminum structural components, cast aluminum engine components, and aluminum sheet metal parts; and lightweight aluminum casings and drive train parts. It also undertakes contract processing for trim pressing, shot blasting, manual processing, heat treatment, truing of structural components, machining, leak testing/impregnation, and washing/pickling passivation. The company was founded in 1996 and is based in Baunatal, Germany. As of January 23, 2017, FINOBA AUTOMOTIVE GmbH operates as a subsidiary of China National Machinery Industry Corporation.	Germany	Europe	EUR 75mn	75	Auto Components
AEZ Leichtmetallräder GmbH	AEZ Leichtmetallräder GmbH manufactures alloy/aluminum wheels for aftermarket sports utility vehicles and off-roaders. It also deals with the repair, cleaning, maintenance, and storage of alloy wheels. The company was founded in 1984 and is based in Siegburg, Germany. AEZ Leichtmetallräder GmbH operates as a subsidiary of Aez Technik Und Design Gmbh.	Germany	Europe	EUR 41mn	41	Auto Components
Elring Klinger (Great Britain) Ltd.	Elring Klinger (Great Britain) Ltd. produces specialty gaskets and shielding parts for the motor vehicle industry. The company was incorporated in 1987 and is based in Redcar, the United Kingdom with a plant in Dettingen/Erms, Germany. Elring Klinger (Great Britain) Ltd. operates as a subsidiary of ElringKlinger AG.	UK	Europe	EUR 39mn	39	Auto Components
FEUER powertrain GmbH & Co. KG	FEUER powertrain GmbH & Co. KG manufactures and sells cast and forged crankshafts. Its products are used in passenger car diesel engines, agricultural harvesters, generator sets, and heavy trucks in Germany and internationally. FEUER powertrain has a strategic partnership with Haowu-Group. The company was founded in 2002 and is based in Nordhausen, Germany. As of June 23, 2017, FEUER powertrain GmbH & Co. KG operates as a subsidiary of Sichuan Haowu Electromechanical Co., Ltd. 	Germany	Europe	EUR 131mn	131	Auto Components
XPEL, Inc.	XPEL, Inc. manufactures, sells, distributes, and installs after-market automotive products in the United States, Canada, the United Kingdom, and the Netherlands. The company offers paint protection films; headlight protection; automotive window films; and plotters. It also provides apparel, merchandise, aftercare products, paint protection films, microfiber products, install tools, and paint protection film install tolls through online. In addition, the company offers installation services and installation training courses; and operates installation centers that serve wholesale and/or retail customers. It sells its products directly to independent installers and new car dealerships, as well as through third party distributors. XPEL, Inc. was founded in 1999 and is headquartered in San Antonio, Texas.	US	United States and Canada	USD 117mn	95	Auto Components
Martinrea Honsel Germany GmbH	Martinrea Honsel Germany GmbH develops and manufactures components for engine, transmission, suspension, and body-in-white of passenger cars and commercial vehicles, as well as for engineering and other applications. The company offers engine blocks, oil pans, and bearing covers for engines; transmission housings; longitudinal beams, swivel bearings, engine cradles, and sub frames for suspensions; and shock towers for body-in-white applications. It also provides switch plates, blind center consoles, and support plates with pressed in grub screw for interior and exterior applications; and aluminum plates and coils for non-automotive applications. Martinrea Honsel Germany GmbH was formerly known as HONSEL AG and changed its name to Martinrea Honsel Germany GmbH in August 2011. The company was founded in 1908 and is based in Meschede, Germany. It has production sites in Germany, Spain, Brazil, Mexico, and China. Martinrea Honsel Germany GmbH operates as a subsidiary of Martinrea International Inc	Germany	Europe	EUR 334mn	334	Auto Components
"Cooper Tire & Rubber Company"	Cooper Tire & Rubber Company, together with its subsidiaries, designs, manufactures, markets, and sells replacement tires in North America, Latin America, Europe, and Asia. The company operates through Americas Tire Operations and International Tire Operations segments. It manufactures and markets passenger car, light truck, truck and bus radial (TBR), motorcycle, and racing tires, as well as tire retread materials; and markets and distributes racing, TBR, and motorcycle tires. The company sells its products to independent tire dealers, wholesale distributors, regional and national retail tire chains, and other tire and automotive product retail chains, as well as original equipment manufacturers; and directly to end users through three owned retail stores. Cooper Tire & Rubber Company was founded in 1914 and is headquartered in Findlay, Ohio."	US	United States and Canada	USD 2772mn	2500	Auto Components
'''

In [5]:
fields = [el.lower().replace(" ", "_")
          for el in
          ("company name	Business Description	Country	Region	Revenue	Converted to EUR mn	Sector / or  SIC Codes"
          .strip("	")
          .split("	")
         )]
annotated_companies = []
for line in base_companies_str.split("\n")[1:]:
    line_dict = {"annotations": {}}
    base_company = {}
    for i,el in enumerate(line.strip("	").split("	")):
        if el.lower() == "not found":
            continue
        if fields[i] == "revenue":
            currency,value = el.split()
            value = float(value[:-2])
            base_company["currency"] = currency
            base_company["revenue_range_pretty"] = f"{currency} {value/10}mn-{value*10}mn"
            continue
        if fields[i] == "converted_to_eur_mn":
            base_company["revenue_range"] = {
                "min": float(el)*1e5,
                "max": float(el)*1e7
            }
            continue
        base_company[fields[i]] = el
    line_dict["base_company"] = base_company
    annotated_companies.append(line_dict)

ValueError: could not convert string to float: 'Stapl'

In [14]:
def search_query(base_name, size=10):
    tst_query_fuzzy  = {
      '_source': True, 
      'from': 0, 
      'size': size, 

      "query": {
          "bool": {
            "should": [
             {
              "multi_match": {
                "query": base_name,
                "fuzziness": "2",
                "prefix_length": 1,
                "fields": [
                  "name",
                  "name.cleaned",
                ],
                "minimum_should_match": "1",
                "type": "most_fields"
              }
             },
            {
              "multi_match": {
                "query": base_name,
                "fuzziness": "1",
                "prefix_length": 1,
                "fields": [
                  "name",
                  "name.cleaned",
                ],
                "minimum_should_match": "1",
                "type": "most_fields",
                "boost": 2
              }
             },
            {
              "multi_match": {
                "query": base_name,
                "fields": [
                  "name",
                  "name.cleaned",
                ],
                "minimum_should_match": "1",
                "type": "most_fields",
                "boost": 4
              }
             }
            ]
           }
          } 
    }

    json_result = client.search(index=INDEX, body=tst_query_fuzzy)
    return [hit["_source"] for hit in json_result["hits"]["hits"]]

In [15]:
Positive, Neutral, Negative = 1,0,-1

In [16]:
model_path = "s3://oaknorth-ml-dev-eu-west-1/andrei/peers/"

In [17]:
[hit["name"] for hit in search_query("Electronic4you GmbH")]

['electronic4you GmbH',
 'Electronic4you Gmbh',
 'RKM GmbH Personaldienstleistungen',
 'BANKPOWER GmbH Personaldienstleistungen',
 'Teamkompetent Gmbh Personaldienstleistungen',
 'teamkompetent GmbH Personaldienstleistungen',
 'Conexa Gmbh Präzisionsarmaturen',
 'Genopersonalconsult Gmbh',
 'Donaldson GmbH',
 'Konzentration GmbH']

In [18]:
for annotated_set in annotated_companies:
    for base_name,metadata in annotated_set["annotations"].items():
        found_name = [hit["name"] for hit in search_query(base_name)][0]
        if found_name != base_name:
            print(f"Failed to find {base_name}, found {found_name} instead.")
        metadata["entity_id"] = [hit["entity_id"] for hit in search_query(base_name)][0]
        print("---------")

In [19]:
import smart_open
import pickle

with smart_open.open(f"{model_path}annotated_peers_batch2.pkl", "wb") as f:
    pickle.dump(annotated_companies, f)